In [ ]:
!pip install PyPDF2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.7 MB/s eta 0:00:00


In [ ]:
import PyPDF2

def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ''
        for page in range(len(reader.pages)):
            text += reader.pages[page].extract_text()
        return text

pdf_file_path = "/content/fecu103.pdf"


extracted_text = extract_text_from_pdf(pdf_file_path)

print(extracted_text)


In [ ]:

def chunk_text(text, chunk_size=512):
    return [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]


chunks = chunk_text(extracted_text)
print(f"Number of chunks: {len(chunks)}")


Number of chunks: 77


In [ ]:
!pip install sentence-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 3.5 MB/s eta 0:00:00


In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np

model = SentenceTransformer('paraphrase-MiniLM-L6-v2')


def generate_embeddings(chunks):
    return model.encode(chunks)


embeddings = generate_embeddings(chunks)


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
!pip install faiss-cpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 45.4 MB/s eta 0:00:00


In [ ]:
import faiss


embedding_size = 384
index = faiss.IndexFlatL2(embedding_size)


index.add(np.array(embeddings))


print(f"Number of entries in the FAISS index: {index.ntotal}")


Number of entries in the FAISS index: 77


In [ ]:

def retrieve_relevant_chunks(query, top_k=5):
    query_embedding = model.encode([query])
    distances, indices = index.search(np.array(query_embedding), top_k)
    relevant_chunks = [chunks[i] for i in indices[0]]
    return relevant_chunks

query = "What is the main topic discussed in the document?"
relevant_text = retrieve_relevant_chunks(query)
print("Relevant Chunks:", relevant_text)


Relevant Chunks: ['.2 What are the Components of Food?\nMedu and Mishti visit the ‘Traditional Food Festival’ organised in their school. The theme of the festival is ‘Eat Healthy, Live Healthy’. \nThe festival features various stalls displaying different \nkinds of traditional dishes. Dr Poshita, a nutritional expert, explains to students that ʻHealth is the Ultimate Wealthʼ. \nChapter 3.indd   39 10-07-2024   17:11:10\nCuriosity | Textbook of Science | Grade 640Let us understand what Dr Poshita means by this \nstatement. \nHave you', 'iseases?\nActivity 3.4: Let us conduct a survey\n \x8bStudy the chart given in Fig. 3.5 to explore the functions \nand sources of various food components. Find out more sources of vitamins and minerals. Also, understand the symptoms of the diseases caused by the lack of these food components.\n \x8bVisit your neighbourhood, interact with people and find out if any individual shows the symptoms listed on the\xa0chart (an investigatory project of this kind

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer


gpt_model = GPT2LMHeadModel.from_pretrained('gpt2')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')


tokenizer.pad_token = tokenizer.eos_token
gpt_model.config.pad_token_id = gpt_model.config.eos_token_id

def generate_response(query, relevant_text, max_new_tokens=150):
    input_text = query + '\n\n' + ' '.join(relevant_text)
    input_ids = tokenizer.encode(input_text, return_tensors='pt', padding=True, truncation=True)

    attention_mask = input_ids.ne(tokenizer.pad_token_id).long()

    output = gpt_model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_new_tokens=max_new_tokens,
        num_return_sequences=1,
        pad_token_id=tokenizer.eos_token_id
    )

    return tokenizer.decode(output[0], skip_special_tokens=True)


query = "What is the main topic discussed in the document?"
relevant_text = ["This document discusses various aspects of health and its impacts "]  # Example relevant text

response = generate_response(query, relevant_text)
print("Generated Response:", response)


Generated Response: What is the main topic discussed in the document?

This document discusses various aspects of health and its impacts  on the health of the population. It is intended to be a comprehensive and comprehensive guide to the health of the population. It is intended to be a comprehensive and comprehensive guide to the health of the population. It is intended to be a comprehensive and comprehensive guide to the health of the population. It is intended to be a comprehensive and comprehensive guide to the health of the population. It is intended to be a comprehensive and comprehensive guide to the health of the population. It is intended to be a comprehensive and comprehensive guide to the health of the population. It is intended to be a comprehensive and comprehensive guide to the health of the population. It is intended to be a comprehensive and comprehensive guide to the health of the population. It is intended to be a


In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import PyPDF2


gpt_model = GPT2LMHeadModel.from_pretrained('gpt2')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')


tokenizer.pad_token = tokenizer.eos_token
gpt_model.config.pad_token_id = gpt_model.config.eos_token_id

def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ''
        for page in range(len(reader.pages)):
            text += reader.pages[page].extract_text()
        return text

# Load the PDF and extract text
pdf_file_path = "/content/fecu103.pdf"
pdf_text = extract_text_from_pdf(pdf_file_path)

def generate_response(query):

    truncated_text = pdf_text[:1500]
    input_text = query + '\n\n' + truncated_text
    input_ids = tokenizer.encode(input_text, return_tensors='pt', padding=True, truncation=True)

    attention_mask = input_ids.ne(tokenizer.pad_token_id).long()

    output = gpt_model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_new_tokens=150,
        num_return_sequences=1,
        pad_token_id=tokenizer.eos_token_id
    )

    return tokenizer.decode(output[0], skip_special_tokens=True)


while True:
    question = input("Ask a question about the document (or type 'exit' to quit): ")
    if question.lower() == 'exit':
        break
    answer = generate_response(question)
    print("Answer:", answer)


Ask a question about the document (or type 'exit' to quit): what are fruits involved in this document
Answer: what are fruits involved in this document

Mindful Eating: A Path  
to a Healthy Body3Chapter
कोऽरुक ्? कोऽरुक ्? कोऽरुक ्? 
हितभ ुक् हितभ ुक् ऋतुभुक्
(सुभाहित)
Who is healthy? Who is healthy? Who is healthy? One 
who eats food that is wholesome, in moderate quantities, 
and appropriate for the season, time, and place.
(Wise saying)
Medu and Mishti read 
‘thought of the day’ on the school noticeboard every day. Today’s thought, 
‘annena jā tāni j ivanti’, 
makes them curious. 
Mishti tells Medu that it is a Sanskrit saying which means ‘food gives life to living beings.’ 
Let us try to understand the significance of this saying.
3.1 What Do We Eat?  
Activity 3.1: Let us record 
All of us eat food every day. Food is an essential component of our daily life. List the food items you have consumed over the week in Table 3.1.
अन्‍ने्‍ ज ाता ह्‍ ज ीव हनत 
annena jātāni ji vanti
Chapt